<div style="text-align: center;">
    <h1>Convolutional Neural Network for Stock Price Prediction</h1>
</div>

# 1. Introduction

This project explores the application of `Convolutional Neural Networks` (CNNs) to forecast the stock price of a company within a specified date range. The primary goal is to predict the future stock price as well as the direction of its movement (`Up` or `Down`) based on historical data. 

Stock price prediction is a complex and challenging task due to the inherent volatility and multitude of factors influencing financial markets. Traditional methods often rely on statistical models, but the rise of deep learning has opened new avenues for more accurate and robust predictions.

In this notebook, we leverage a `CNN` model, typically used in image processing, to capture and analyze temporal patterns in stock price data. The model is trained to learn from past price movements and make predictions about future prices and their directional shifts. We also evaluate the model's performance through various metrics, including, but not limited to, `mean percentage error` for the price forecast and a `confusion matrix` for the predicted price movements.

By the end of this notebook, you will gain insights into the effectiveness of CNNs in stock price prediction and understand the model's capability to anticipate price movements, which can be crucial for making informed investment decisions.

In [1]:
import lib
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split

# 2. Data Fetching 

We begin by importing historical stock prices for a specific company using its `ticker`, within a defined date range between `start_date` and `end_date`. This data forms the foundation for our predictive modeling process. 

Once the data is fetched, we prepare it for the CNN model using a custom function, `prepare_data`. This function processes the data by generating sliding windows of `Close` prices, where each window contains a sequence of stock prices over a specified `window_size` (default is 10). These windows act as the input features for our model, while the subsequent stock prices serve as the target labels.

The data is then normalized using a `StandardScaler` to ensure that the CNN model can efficiently learn patterns without being influenced by the scale of the input values. After normalization, the data is reshaped into the appropriate format required by the CNN model: Each sample is structured as a 3D array with dimensions corresponding to the window size, number of features (in this case, one), and a single channel.

This preparation step is crucial as it transforms raw stock price data into a structured format that the CNN can effectively process, setting the stage for accurate and meaningful stock price predictions.

In [2]:
ticker='AOS'
start_date='2020-01-01'
end_date='2024-01-01'

df = lib.load_data(ticker, start_date, end_date)
data, labels = lib.prepare_data(df)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
test_date_range = date_range[-200:]

[*********************100%***********************]  1 of 1 completed


# 3. Model Building 

For predicting stock prices, we utilize a CNN built using the `build_model` function. This model is constructed with a sequential architecture designed to effectively capture patterns in time series data. The network begins with a 2D convolutional layer (`Conv2D`) that extracts features from the input data, followed by a max pooling layer (`MaxPooling2D`) that reduces the spatial dimensions while retaining the most significant features. This process is repeated with an additional convolutional layer, further increasing the depth and complexity of the feature extraction. 

The feature maps produced by the convolutional and pooling layers are then flattened into a 1D vector, which is passed through a dense layer with 512 units, enabling the model to learn complex relationships between features. The final output layer consists of a single neuron that produces the predicted stock price. 

With a total of 40,129 parameters, all of which are trainable, the model is equipped to handle the intricacies of stock price prediction. This architecture allows for effective learning and generalization from historical price data all while keeping manageable complexity and depth.

In [3]:
model = lib.build_model((X_train.shape[1], X_train.shape[2], X_train.shape[3]))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 1, 32)          128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 1, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 1, 64)          6208      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 512)               3

# 4. Model Fitting & Training

To train our CNN model for stock price prediction, we set up a training process spanning 100 epochs with a batch size of 32. The model was trained on the training dataset, and 20% of the data was used for validation to monitor performance on unseen data.

We incorporated two key callbacks to enhance the training process: `Early Stopping` and `Learning Rate Reduction`. Early Stopping was used to halt training if the model's performance on the validation set did not improve for a specified number of epochs, preventing overfitting and saving computational resources. Learning Rate Reduction dynamically adjusted the learning rate based on the model's performance, ensuring more stable and effective convergence.

In [4]:
callbacks = lib.get_callbacks() # Early Stopping & Learning Rate Reduction

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=0,
    callbacks=callbacks
)

loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.3f}')

7/7 [==============================] - 0s 2ms/step - loss: 4.6877
Test Loss: 4.688


# 5. Predictions

In this section, we make predictions on the test set and we generate two key outputs: the `predicted stock price values` and the `predicted stock price movements`: `Up` (Green) or `Down` (Red), compared to the previous day. The ***predicted stock price values*** provide a continuous output, reflecting the model's estimate of the stock's closing price for each day in the test period. These predictions are crucial for assessing the overall trend and accuracy of the model in capturing the underlying stock price dynamics.

The second type of prediction involves determining the ***daily price movement***, categorized as up, down, or unchanged relative to the previous day's closing price. This binary classification task is vital for understanding the model's capability to capture the directional trends of the stock. This dual prediction approach allows for a comprehensive assessment of the model's performance in both forecasting price levels and capturing the directional shifts of the stock.

In [5]:
predictions_df = lib.run_predictions(model, X_test, y_test, test_date_range)

lib.mean_percentage_error(predictions_df)

predictions_df.head()

7/7 [==============================] - 0s 1ms/step


# 6. Model Evaluating & Monitoring

In this section, we focus on evaluating and monitoring the performance of our `CNN` model. We'll explore various `metrics` to assess how well the model predicts stock prices and their movements.

In [10]:
lib.plot_monitoring_dashboard(history, predictions_df, ticker)

In [13]:
print("*** Mean Percentage Error Distribution Parameters ***")

mean = np.mean(predictions_df['MPE'])
std = np.std(predictions_df['MPE'])
skewe = skew(predictions_df['MPE'])
kurt = kurtosis(predictions_df['MPE'])

print(f"Mean = {mean:.2f}% | Standard Deviation = {std:.2f}% | Skewness = {skewe:.3f} | Kurtosis = {kurt:.3f}")

*** Mean Percentage Error Distribution Parameters ***
Mean = 0.23% | Standard Deviation = 3.71% | Skewness = -1.795 | Kurtosis = 7.275


The Mean Percentage Error analysis reveals that the model's predictions are generally unbiased, as indicated by the `mean` MPE being close to zero. This suggests that, on average, the model neither systematically overestimates nor underestimates stock prices. The `standard deviation` of around 4% reflects moderate variability in prediction accuracy, implying that while the model performs well overall, there are occasional fluctuations in prediction errors. The negative `skewness` of the MPE indicates a tendency for the model to overestimate stock prices more frequently than it underestimates them. Additionally, the high `kurtosis` suggests that the distribution of errors has heavy tails and a sharper peak compared to a normal distribution, indicating the presence of occasional large prediction errors.

In [12]:
print("*** Binary Price Movements Predictions Metrics ***")

cm, f1, acc = lib.binary_evaluation_metrics(predictions_df)
print("Confusion Matrix:")
display(cm)
print(f"F1 Score = {f1*100:.2f}% | Accuracy Score = {acc*100:.2f}%")

*** Binary Price Movements Predictions Metrics ***
Confusion Matrix:


,Predicted Up,Predicted Down
Actual Up,87,7
Actual Down,10,94


F1 Score = 91.10% | Accuracy Score = 91.41%


The `confusion matrix` indicates that the model correctly predicted a high number of `Up` and `Down` movements, with only a few false `Up` and false `Down`. This results in an `accuracy` of above 90% and an `F1 score` of more than 90%. These metrics suggest that the model performs well, with a good balance between precision and recall, making it reliable for predicting stock price movements.

# 7. Conclusion

In conclusion, this project demonstrates the application of a `CNN` model for stock price prediction, leveraging historical data to forecast both the `stock price` and the `directional movement` (up or down) on a daily basis. Throughout the notebook, we explored the entire workflow, from `data fetching` and `preparation` to `model building`, `training`, and `evaluation`.

The results indicate that while the model performs `reasonably well` in predicting stock prices and movements, certain `limitations` remain, particularly in capturing extreme market events or unusual price fluctuations. The use of `early stopping` and `learning rate reduction` helped to optimize the model's performance, ensuring that the model did not overfit and could generalize better to unseen data.

Overall, this project provides a strong foundation for `further refinement` and enhancement of predictive models in the financial domain, highlighting the potential of deep learning techniques in stock market analysis and prediction. Future work could focus on integrating more sophisticated models, additional features, or alternative data sources to improve prediction accuracy and robustness.